In [ ]:
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ===================== Fase de Monitoramento =====================
def monitoramento(dados):
    """
    Fase de monitoramento baseada em regras simples.
    Avalia as métricas para identificar tendências anormais.
    """
    print("===== Fase de Monitoramento =====")
    for dado in dados:
        tempo_resposta, utilizacao_cpu = dado
        if tempo_resposta > 1.0:
            print(f"Alerta: Tempo de resposta elevado ({tempo_resposta}s).")
        if utilizacao_cpu > 0.7:
            print(f"Alerta: Alta utilização da CPU ({utilizacao_cpu * 100:.1f}%).")
    return dados

# Dados simulados para monitoramento
dados_monitorados = [
    [0.5, 0.3], [1.2, 0.8], [0.6, 0.4], [1.1, 0.9], [0.3, 0.2],
    [0.8, 0.5], [1.5, 0.7], [0.4, 0.3], [1.3, 0.8], [0.9, 0.6]
]

# Fase de monitoramento
dados_filtrados = monitoramento(dados_monitorados)

# ===================== Fase de Análise =====================
# Dados históricos para treinamento (incluindo labels "Boa" (0) ou "Ruim" (1))
dados_treino = np.array([[1.73, 0.95, 1.0],
 [1.56, 0.78, 0.0],
 [0.26, 0.78, 0.0],
 [1.02, 0.57, 0.0],
 [1.73, 0.95, 1.0],
 [1.93, 0.57, 0.0],
 [1.06, 0.65, 1.0],
 [0.37, 0.74, 0.0],
 [1.03, 0.56, 0.0],
 [0.72, 0.45, 0.0],
 [0.79, 0.92, 0.0],
 [0.39, 0.85, 0.0],
 [2.26, 0.45, 1.0],
 [1.74, 0.85, 1.0],
 [0.93, 0.06, 0.0],
 [2.24, 0.85, 1.0],
 [2.33, 0.64, 1.0],
 [1.32, 0.12, 0.0],
 [0.43, 0.79, 1.0],
 [1.43, 0.37, 0.0],
 [1.42, 0.61, 0.0],
 [0.84, 1.0, 0.0],
 [1.36, 0.77, 1.0],
 [0.28, 0.97, 0.0],
 [1.18, 0.68, 1.0],
 [2.21, 0.78, 1.0],
 [0.54, 0.97, 0.0],
 [2.29, 0.06, 0.0],
 [0.47, 0.16, 0.0],
 [1.8, 0.35, 0.0],
 [2.25, 0.85, 1.0],
 [1.62, 0.55, 0.0],
 [0.83, 0.53, 0.0],
 [0.5, 0.94, 0.0],
 [1.68, 0.72, 1.0],
 [2.12, 0.56, 0.0],
 [0.17, 0.05, 0.0],
 [1.48, 0.65, 1.0],
 [1.1, 0.64, 1.0],
 [1.61, 0.29, 0.0],
 [1.26, 0.8, 1.0],
 [0.81, 0.07, 1.0],
 [1.16, 0.48, 0.0],
 [1.56, 0.31, 0.0],
 [1.86, 0.86, 1.0],
 [0.41, 0.37, 0.0],
 [0.87, 0.47, 0.0],
 [0.7, 0.1, 0.0],
 [1.02, 0.62, 1.0],
 [2.18, 0.16, 0.0],
 [1.49, 0.31, 0.0],
 [1.3, 0.51, 0.0],
 [0.84, 0.02, 0.0],
 [1.31, 0.97, 1.0],
 [0.95, 0.05, 0.0],
 [2.22, 0.65, 1.0],
 [1.39, 0.85, 1.0],
 [2.22, 0.73, 1.0],
 [0.98, 0.4, 0.0],
 [0.57, 0.55, 1.0],
 [1.31, 0.76, 1.0],
 [2.47, 0.68, 1.0],
 [2.44, 0.39, 0.0],
 [0.91, 0.94, 0.0],
 [0.58, 0.51, 0.0],
 [0.21, 0.14, 0.0],
 [1.24, 0.46, 0.0],
 [1.34, 0.33, 0.0],
 [0.49, 0.99, 0.0],
 [0.82, 0.34, 0.0],
 [1.38, 0.71, 1.0],
 [2.06, 0.37, 0.0],
 [2.45, 0.58, 0.0],
 [1.84, 0.69, 0.0],
 [1.24, 0.97, 1.0],
 [1.96, 0.58, 0.0],
 [1.5, 0.17, 0.0],
 [1.59, 0.84, 1.0],
 [1.73, 0.03, 0.0],
 [0.36, 0.02, 0.0],
 [0.46, 0.69, 0.0],
 [1.96, 0.92, 1.0],
 [1.87, 0.06, 0.0],
 [0.6, 0.33, 0.0],
 [1.36, 0.31, 0.0],
 [2.29, 0.34, 0.0],
 [1.95, 0.72, 1.0],
 [1.76, 0.61, 1.0],
 [0.69, 0.56, 0.0],
 [2.43, 0.3, 0.0],
 [0.6, 0.7, 0.0],
 [0.94, 0.93, 0.0],
 [0.76, 0.12, 0.0],
 [1.01, 0.98, 1.0],
 [2.39, 0.8, 1.0],
 [0.79, 0.71, 0.0],
 [1.16, 0.26, 0.0],
 [0.58, 0.54, 0.0],
 [1.77, 0.14, 0.0],
 [1.51, 0.24, 0.0],
 [1.37, 0.64, 0.0],
 [1.09, 0.72, 1.0],
 [1.95, 0.01, 0.0],
 [2.33, 0.41, 0.0],
 [2.21, 0.48, 0.0],
 [2.41, 0.32, 0.0],
 [2.28, 0.09, 0.0],
 [1.38, 0.72, 1.0],
 [2.29, 0.18, 0.0],
 [0.52, 0.92, 0.0],
 [1.05, 0.5, 0.0],
 [2.41, 0.93, 1.0],
 [0.12, 0.57, 0.0],
 [2.46, 0.28, 0.0],
 [1.4, 0.49, 0.0],
 [2.14, 0.47, 0.0],
 [0.37, 0.16, 0.0],
 [0.72, 0.19, 0.0],
 [2.0, 0.57, 0.0],
 [0.52, 0.86, 0.0],
 [2.08, 0.52, 1.0],
 [1.71, 0.18, 0.0],
 [0.88, 0.25, 0.0],
 [0.67, 0.75, 0.0],
 [0.82, 0.72, 1.0],
 [1.67, 0.69, 0.0],
 [0.38, 0.31, 0.0],
 [1.31, 0.9, 1.0],
 [0.85, 0.12, 0.0],
 [0.81, 0.61, 0.0],
 [0.42, 0.15, 0.0],
 [1.55, 0.42, 0.0],
 [1.23, 0.54, 0.0],
 [0.62, 0.25, 0.0],
 [2.2, 0.08, 0.0],
 [1.79, 0.08, 0.0],
 [0.25, 0.55, 0.0],
 [1.47, 0.15, 0.0],
 [1.35, 0.12, 0.0],
 [1.5, 0.09, 0.0],
 [2.04, 0.45, 0.0],
 [1.2, 0.06, 0.0],
 [2.04, 0.72, 1.0],
 [2.06, 0.75, 1.0],
 [0.21, 0.34, 1.0],
 [2.49, 0.93, 0.0],
 [2.34, 0.03, 0.0],
 [1.95, 0.77, 1.0],
 [1.65, 0.42, 0.0],
 [1.02, 0.87, 1.0],
 [1.0, 0.68, 0.0],
 [1.39, 0.65, 1.0],
 [0.53, 0.54, 0.0],
 [1.85, 0.22, 1.0],
 [0.15, 0.3, 0.0],
 [1.41, 0.53, 0.0],
 [0.69, 0.35, 0.0],
 [2.35, 0.73, 1.0],
 [2.04, 0.42, 0.0],
 [2.22, 0.02, 0.0],
 [0.34, 0.03, 0.0],
 [1.79, 0.05, 0.0],
 [1.07, 0.23, 0.0],
 [2.17, 0.06, 0.0],
 [0.79, 0.82, 0.0],
 [0.87, 0.6, 0.0],
 [0.87, 0.3, 0.0],
 [1.68, 0.22, 0.0],
 [0.25, 0.95, 0.0],
 [2.29, 0.63, 1.0],
 [1.29, 0.97, 1.0],
 [1.71, 0.79, 1.0],
 [1.1, 0.15, 0.0],
 [1.91, 0.47, 0.0],
 [0.82, 0.71, 0.0],
 [2.3, 0.56, 0.0],
 [1.44, 0.13, 0.0],
 [0.59, 0.65, 0.0],
 [2.13, 0.09, 0.0],
 [0.56, 0.27, 0.0],
 [1.55, 0.87, 1.0],
 [1.93, 0.72, 1.0],
 [1.25, 0.87, 1.0],
 [2.4, 0.02, 0.0],
 [2.41, 0.12, 0.0],
 [1.25, 0.24, 0.0],
 [0.59, 0.92, 0.0],
 [1.96, 0.38, 0.0],
 [0.96, 0.26, 0.0],
 [1.29, 0.1, 0.0],
 [1.23, 0.84, 1.0],
 [0.99, 0.41, 0.0],
 [0.63, 0.15, 0.0],
 [2.34, 0.58, 0.0],
 [0.47, 0.33, 0.0],
 [2.1, 0.5, 0.0],
 [1.74, 0.26, 0.0],
 [2.42, 0.64, 1.0],
 [0.5, 0.79, 0.0],
 [1.83, 0.49, 0.0],
 [1.4, 0.64, 0.0],
 [0.18, 0.85, 0.0],
 [1.7, 0.76, 1.0],
 [2.12, 0.23, 0.0],
 [0.12, 0.51, 0.0],
 [1.58, 0.67, 1.0],
 [1.26, 0.49, 1.0],
 [1.42, 0.01, 0.0],
 [0.76, 0.98, 1.0],
 [2.05, 0.67, 1.0],
 [2.28, 0.11, 1.0],
 [0.46, 0.19, 0.0],
 [2.06, 0.27, 0.0],
 [1.0, 0.41, 0.0],
 [2.48, 0.23, 0.0],
 [2.13, 0.65, 1.0],
 [1.92, 0.09, 0.0],
 [1.43, 0.06, 1.0],
 [0.51, 0.5, 0.0],
 [1.98, 0.57, 0.0],
 [0.33, 0.53, 1.0],
 [0.61, 0.87, 0.0],
 [1.24, 0.05, 1.0],
 [2.32, 0.9, 1.0],
 [0.18, 0.93, 0.0],
 [2.41, 0.59, 0.0],
 [1.81, 0.4, 1.0],
 [0.49, 0.24, 0.0],
 [1.03, 0.9, 1.0],
 [1.91, 0.14, 0.0],
 [1.84, 0.5, 0.0],
 [0.23, 0.44, 0.0],
 [0.92, 0.77, 0.0],
 [1.05, 0.77, 0.0],
 [0.76, 0.99, 0.0],
 [0.95, 0.94, 0.0],
 [1.73, 0.66, 0.0],
 [1.58, 0.8, 1.0],
 [0.3, 0.15, 0.0],
 [1.62, 0.74, 1.0],
 [0.36, 0.01, 0.0],
 [0.96, 0.27, 0.0],
 [0.55, 0.45, 0.0],
 [1.08, 0.03, 0.0],
 [0.32, 0.6, 0.0],
 [1.02, 0.29, 0.0],
 [0.3, 0.9, 0.0],
 [2.45, 0.57, 0.0],
 [1.01, 0.14, 0.0],
 [1.28, 0.06, 0.0],
 [1.11, 0.48, 1.0],
 [0.7, 0.25, 0.0],
 [1.53, 0.2, 0.0],
 [0.83, 0.95, 0.0],
 [1.59, 0.8, 1.0],
 [0.9, 0.82, 0.0],
 [2.44, 0.14, 0.0],
 [2.37, 0.2, 0.0],
 [2.41, 0.97, 1.0],
 [1.77, 0.49, 0.0],
 [0.68, 0.38, 0.0],
 [1.04, 0.11, 0.0],
 [1.52, 0.55, 0.0],
 [1.42, 0.95, 1.0],
 [1.8, 0.44, 0.0],
 [1.76, 0.82, 1.0],
 [1.08, 0.5, 0.0],
 [0.68, 0.66, 0.0],
 [1.99, 0.07, 0.0],
 [1.25, 0.4, 0.0],
 [2.31, 0.69, 1.0],
 [2.0, 0.36, 0.0],
 [1.39, 0.64, 0.0],
 [1.95, 0.66, 1.0],
 [1.65, 0.04, 0.0],
 [1.73, 0.4, 0.0],
 [2.42, 0.43, 1.0],
 [0.72, 0.51, 0.0],
 [1.49, 0.58, 0.0],
 [1.04, 0.77, 1.0],
 [1.3, 0.34, 1.0],
 [0.35, 0.42, 0.0],
 [0.38, 0.94, 0.0],
 [0.85, 0.46, 0.0],
 [1.26, 0.48, 0.0],
 [1.77, 0.32, 0.0],
 [2.04, 0.12, 0.0],
 [1.66, 0.68, 1.0],
 [2.46, 0.24, 0.0],
 [0.48, 0.56, 0.0],
 [0.66, 0.41, 0.0],
 [1.64, 0.43, 1.0],
 [1.57, 0.2, 0.0],
 [1.03, 0.7, 1.0],
 [1.91, 0.81, 0.0],
 [0.34, 0.87, 0.0],
 [0.88, 0.46, 0.0],
 [2.17, 0.53, 0.0],
 [1.53, 0.61, 1.0],
 [0.94, 0.85, 0.0],
 [2.05, 0.71, 1.0],
 [1.57, 0.08, 0.0],
 [0.38, 0.31, 0.0],
 [1.76, 0.51, 0.0],
 [0.43, 0.38, 0.0],
 [1.63, 0.69, 1.0],
 [2.5, 0.55, 0.0],
 [0.44, 0.31, 0.0],
 [0.23, 0.36, 1.0],
 [0.43, 0.82, 0.0],
 [1.31, 0.49, 0.0],
 [1.1, 0.84, 1.0],
 [0.3, 0.03, 0.0],
 [0.8, 0.27, 0.0],
 [0.5, 0.46, 0.0],
 [1.94, 0.55, 0.0],
 [0.37, 0.78, 0.0],
 [0.98, 0.82, 1.0],
 [1.83, 0.55, 0.0],
 [0.35, 0.83, 0.0],
 [0.81, 1.0, 0.0],
 [0.94, 0.82, 0.0],
 [2.49, 0.78, 1.0],
 [2.05, 0.59, 0.0],
 [1.81, 0.63, 1.0],
 [0.43, 0.21, 0.0],
 [0.24, 0.35, 0.0],
 [1.39, 0.32, 0.0],
 [0.79, 0.27, 0.0],
 [2.47, 0.68, 0.0],
 [2.41, 0.79, 1.0],
 [2.48, 0.87, 1.0],
 [2.18, 0.25, 0.0],
 [2.09, 0.76, 1.0],
 [1.27, 0.58, 0.0],
 [1.09, 0.45, 0.0],
 [2.21, 0.09, 0.0],
 [0.77, 0.94, 0.0],
 [2.38, 0.33, 1.0],
 [1.96, 0.73, 1.0],
 [1.2, 0.66, 1.0],
 [0.25, 0.53, 0.0],
 [1.13, 0.21, 0.0],
 [2.09, 0.34, 0.0],
 [1.46, 0.49, 0.0],
 [1.74, 0.05, 1.0],
 [1.98, 0.46, 0.0],
 [2.16, 0.44, 1.0],
 [2.4, 0.2, 0.0],
 [0.42, 0.65, 0.0],
 [2.34, 0.27, 0.0],
 [0.7, 0.37, 0.0],
 [0.5, 0.4, 0.0],
 [1.78, 0.23, 0.0],
 [1.79, 0.0, 0.0],
 [0.42, 0.23, 0.0],
 [0.12, 0.7, 0.0],
 [2.47, 0.42, 0.0],
 [0.27, 0.38, 0.0],
 [0.35, 0.31, 0.0],
 [0.43, 0.77, 0.0],
 [0.42, 0.99, 0.0],
 [1.37, 0.01, 0.0],
 [1.16, 0.72, 1.0],
 [0.46, 0.41, 0.0],
 [2.16, 0.09, 0.0],
 [1.91, 0.59, 0.0],
 [2.41, 0.31, 0.0],
 [0.76, 0.08, 0.0],
 [1.54, 0.61, 1.0],
 [1.76, 0.85, 1.0],
 [0.82, 0.52, 0.0],
 [1.89, 0.3, 1.0],
 [2.25, 0.95, 1.0],
 [0.37, 0.5, 0.0],
 [1.37, 0.98, 1.0],
 [2.34, 0.13, 0.0],
 [1.46, 0.37, 0.0],
 [1.93, 0.95, 1.0],
 [0.14, 0.07, 0.0],
 [0.2, 0.06, 0.0],
 [1.32, 0.63, 1.0],
 [1.1, 0.7, 0.0],
 [1.39, 0.62, 1.0],
 [0.84, 0.51, 0.0],
 [2.04, 0.54, 0.0],
 [1.62, 0.83, 1.0],
 [0.26, 0.7, 0.0],
 [2.13, 0.06, 1.0],
 [1.14, 0.45, 0.0],
 [0.84, 0.74, 0.0],
 [0.39, 0.71, 0.0],
 [0.49, 0.95, 0.0],
 [1.98, 0.72, 1.0],
 [0.4, 0.78, 0.0],
 [2.23, 0.77, 0.0],
 [2.04, 0.27, 1.0],
 [1.81, 0.58, 1.0],
 [1.19, 0.36, 0.0],
 [1.46, 0.37, 0.0],
 [0.35, 0.57, 0.0],
 [0.65, 0.51, 1.0],
 [2.17, 0.24, 0.0],
 [1.02, 0.15, 0.0],
 [2.16, 0.55, 0.0],
 [1.88, 0.42, 0.0],
 [1.1, 0.72, 1.0],
 [0.29, 0.37, 0.0],
 [2.26, 0.18, 0.0],
 [2.46, 0.95, 0.0],
 [1.22, 0.28, 0.0],
 [0.89, 0.55, 1.0],
 [0.58, 0.56, 0.0],
 [1.59, 0.46, 0.0],
 [1.28, 0.77, 1.0],
 [2.26, 0.76, 1.0],
 [0.12, 0.41, 0.0],
 [0.93, 0.84, 0.0],
 [2.38, 0.0, 0.0],
 [2.14, 0.73, 1.0],
 [1.37, 0.24, 0.0],
 [0.24, 1.0, 0.0],
 [0.32, 0.92, 0.0],
 [1.35, 0.7, 1.0],
 [2.44, 0.08, 1.0],
 [0.42, 0.82, 1.0],
 [1.46, 0.43, 0.0],
 [0.67, 0.26, 0.0],
 [2.02, 0.7, 1.0],
 [0.86, 0.27, 1.0],
 [0.59, 0.78, 0.0],
 [0.47, 0.16, 0.0],
 [1.08, 0.54, 0.0],
 [0.6, 0.31, 0.0],
 [0.39, 0.16, 0.0],
 [2.08, 0.7, 0.0],
 [2.11, 0.33, 1.0],
 [0.7, 0.36, 0.0],
 [1.73, 0.26, 0.0],
 [0.17, 0.4, 0.0],
 [1.9, 0.25, 0.0],
 [2.03, 0.14, 1.0],
 [2.09, 0.98, 1.0],
 [2.08, 0.37, 0.0],
 [1.65, 0.58, 0.0],
 [2.05, 0.02, 1.0],
 [2.27, 0.44, 0.0],
 [2.27, 0.83, 1.0],
 [0.2, 0.46, 0.0],
 [1.48, 0.82, 1.0],
 [0.17, 0.68, 0.0],
 [0.62, 0.19, 0.0],
 [2.22, 0.03, 0.0],
 [0.69, 0.3, 0.0],
 [1.42, 0.06, 0.0],
 [0.43, 0.2, 0.0],
 [1.36, 0.63, 1.0],
 [2.01, 0.99, 1.0],
 [0.96, 0.54, 0.0],
 [2.29, 0.5, 0.0],
 [0.53, 0.32, 0.0],
 [2.25, 0.78, 0.0],
 [2.48, 0.53, 0.0],
 [2.5, 0.97, 1.0],
 [1.68, 0.27, 0.0],
 [2.3, 0.06, 0.0],
 [0.63, 0.85, 0.0],
 [0.95, 0.84, 0.0],
 [0.52, 0.59, 0.0],
 [1.77, 0.91, 0.0],
 [1.78, 0.95, 1.0],
 [1.45, 0.19, 0.0],
 [2.22, 0.49, 0.0],
 [1.28, 0.09, 0.0],
 [0.63, 0.53, 1.0],
 [2.3, 0.2, 0.0],
 [1.82, 0.92, 1.0],
 [0.88, 0.02, 0.0],
 [2.3, 0.77, 1.0],
 [2.17, 0.96, 1.0],
 [2.36, 0.4, 0.0],
 [0.82, 0.14, 0.0],
 [2.38, 0.79, 1.0],
 [1.66, 0.17, 0.0],
 [1.76, 0.93, 1.0],
 [0.64, 0.59, 0.0],
 [0.84, 0.69, 0.0],
 [1.85, 0.95, 1.0],
 [1.04, 0.99, 1.0],
 [0.18, 0.6, 0.0],
 [2.42, 0.22, 0.0],
 [2.35, 0.14, 0.0],
 [0.67, 0.98, 0.0],
 [2.22, 0.09, 0.0],
 [1.63, 0.89, 1.0],
 [0.74, 0.25, 1.0],
 [0.72, 0.11, 1.0],
 [1.03, 0.73, 1.0],
 [2.22, 0.49, 0.0],
 [1.41, 0.1, 0.0]])
X = dados_treino[:, :-1]
y = dados_treino[:, -1]

# Divisão dos dados e normalização
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Balanceamento de classes
weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(weights))

# Configuração da rede neural
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Adicionando EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

print("\n===== Fase de Análise =====")
start_time = time.time()
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    class_weight=class_weights,
    callbacks=[early_stopping],
    verbose=0
)
training_time = time.time() - start_time

# Avaliação do modelo
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
y_pred = model.predict(X_test).flatten()
y_pred_classes = (y_pred >= 0.5).astype(int)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Acurácia: {accuracy:.2f}")
print(f"Loss: {loss:.2f}")
print(f"ROC-AUC: {roc_auc:.2f}")
print(f"Tempo de treinamento: {training_time:.2f} segundos")
print(classification_report(y_test, y_pred_classes, target_names=["Good", "Bad"]))

# Matriz de confusão
cm = confusion_matrix(y_test, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Good", "Bad"])
disp.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix")
plt.show()

# ===================== Fase de Planejamento =====================
def planejar_acao(predicao, cpu_usage):
    """
    Planeja ações mais específicas dependendo da previsão e utilização de CPU.
    """
    if predicao >= 0.5:
        if cpu_usage > 0.8:
            return "Redistribuir tarefas críticas e alocar recursos extras."
        else:
            return "Monitorar de perto a carga para evitar sobrecarga futura."
    #fazer aqui a divisão de análise e planejamento, que até então não está corretamente feito
    return "Nenhuma ação necessária."

print("\n===== Fase de Planejamento =====")
acoes = []
planejamento_start = time.time()
for i, predicao in enumerate(y_pred):
    estado_antes = {"tempo_resposta": X_test[i][0], "uso_cpu": X_test[i][1]}
    acao = planejar_acao(predicao, estado_antes["uso_cpu"])
    estado_depois = {"tempo_resposta": estado_antes["tempo_resposta"] * 0.85}
    #esta é uma possibilidade de melhoria
    #sucesso = (estado_depois["tempo_resposta"] < estado_antes["tempo_resposta"]) and (estado_antes["uso_cpu"] < 0.7)
    #o de baixo é o original
    sucesso = ((estado_depois["tempo_resposta"] < estado_antes["tempo_resposta"]) or (estado_antes["uso_cpu"] > 0.7))
    acoes.append({
        "Entrada": estado_antes,
        "Situação": "Ruim" if predicao >= 0.5 else "Boa",
        "Ação": acao,
        "Sucesso": sucesso
    })
planejamento_time = time.time() - planejamento_start

sucessos = sum(acao["Sucesso"] for acao in acoes)
taxa_sucesso = sucessos / len(acoes) * 100
print(f"Latência do planejamento: {planejamento_time:.2f} segundos")
print(f"Taxa de sucesso das ações: {taxa_sucesso:.2f}%")

# ===================== Fase de Conhecimento =====================
print("\n===== Fase de Conhecimento =====")
repositorio_conhecimento = []

#isso não é uso efetivo. Estou apenas analisando o quanto estou guardando no repositório para posterior análise
for acao in acoes:
    acao["Impacto"] = "Melhora significativa" if acao["Sucesso"] else "Nenhuma melhora"
    repositorio_conhecimento.append(acao)

#uso_efetivo = sum(1 for acao in repositorio_conhecimento if acao["Sucesso"]) / len(repositorio_conhecimento) * 100
uso_efetivo = sum(1 for acao in repositorio_conhecimento if acao["Ação"] != "Nenhuma") / len(repositorio_conhecimento) * 100
print(f"Uso efetivo do conhecimento: {uso_efetivo:.2f}%")

# Exibindo algumas ações
for entrada in repositorio_conhecimento[:10]:  # Limitando para os primeiros 10
    print(entrada)
